In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/TimeLineChart.ipynb

Selectors :
   * absoluteToRelativeCompletableProgress : checkboxe (T/F) value to display the line chart per relative time

In [ ]:
#constructing a checkboxe to view chart per relative time
absoluteToRelativeCompletableProgress=widgets.Checkbox(value=False,
                                        description='View in relative time')

***vis_completables_progress_increase_decrease* function**:
* Output : the outputCompletableProgress in a list to be display in a VBox
* outputCompletableProgressIncreaseDecrease : display a points/line chart showing for each player in function of time the progress increase or decrease of different completables of the game

In [ ]:
def vis_completables_progress_increase_decrease():
    def display_completable_progress_increase_decrease(change):
        with outputCompletableProgressIncreaseDecrease:
            clear_output(wait=False)
            getProgressPlayerIncreaseDecreasePerCompletable()
            clear_output(wait=True)
    absoluteToRelativeCompletableProgress.observe(display_completable_progress_increase_decrease,'value')
    display_completable_progress_increase_decrease(None)
    return [absoluteToRelativeCompletableProgress, outputCompletableProgressIncreaseDecreaseUpdate, outputCompletableProgressIncreaseDecrease]

***getProgressPlayerIncreaseDecreasePerCompletable* function**:
Output : displays a points/line chart showing for each player in function of time the progress increase or decrease of different completables of the game

In [ ]:
def getProgressPlayerIncreaseDecreasePerCompletable():
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    completableSelectedProgress=\
        getSublistSelectedFromUnionListSelected(completables_increase_decrease_list,
                                                checkboxesCompletableSelected.get_selected_options())
    if len(completableSelectedProgress)>0 and len(playersSelected)>0:
        accordionCompletableProgressIncreaseDecrease=\
            widgets.Accordion(children=[widgets.Output() for i in range(len(completableSelectedProgress))],
                              selected_index=None)
        i=0
        for completable in completableSelectedProgress:
            accordionCompletableProgressIncreaseDecrease.set_title(i, completable)
            i+=1
        display(accordionCompletableProgressIncreaseDecrease)
        def on_selected_index_completable_progress_increase_decrease_change(change):
            index=change['new']
            if index!=None:
                #close and clear previous fig of completable progress
                for fig in figCompletableProgressIncreaseDecrease:
                    plt.close(fig=fig)
                figCompletableProgressIncreaseDecrease.clear()
                playersSelected=checkboxesPlayersSelected.get_selected_options()
                completableSelectedProgress=\
                    getSublistSelectedFromUnionListSelected(completables_increase_decrease_list,
                                                            checkboxesCompletableSelected.get_selected_options())
                completable=completableSelectedProgress[index]
                with accordionCompletableProgressIncreaseDecrease.children[index]:
                    clear_output(wait=False)
                    #create new fig with custom name in the tab
                    name=os.path.splitext(filename)[0]+"_completable_progress_increase_decrease_"+completable
                    fig = plt.figure(num=name, constrained_layout=True)
                    fig.canvas.header_visible = False
                    fig.canvas.layout.min_height = '400px'
                    figCompletableProgressIncreaseDecrease.append(fig)
                    completableProgressIncreaseDecrease={}
                    for player in playersSelected:
                        if completable in players_info[player]["completables_progress"].keys():
                            completableProgressIncreaseDecrease[player]={}
                            for tabValTime in players_info[player]["completables_progress"][completable]:
                                completableProgressIncreaseDecrease[player][tabValTime[1]]=tabValTime[0]
                            completableProgressIncreaseDecrease[player]=dict(sorted(completableProgressIncreaseDecrease[player].items(), key=lambda t: t[0]))
                            i=0
                            for key,value in completableProgressIncreaseDecrease[player].items():
                                if i==0:
                                    previousval=value
                                else:
                                    previousval+=value
                                    completableProgressIncreaseDecrease[player][key]=previousval
                                i+=1
                    for player in completableProgressIncreaseDecrease.keys():
                        # one line chart per player showing progress over time
                        y = list(completableProgressIncreaseDecrease[player].values())
                        if absoluteToRelativeCompletableProgress.value:
                            x = absoluteToRelativeTime(list(completableProgressIncreaseDecrease[player].keys()))
                        else:
                            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S %d-%m-%y'))
                            locator=mdates.MinuteLocator(interval=10)
                            locator.MAXTICKS=40000
                            plt.gca().xaxis.set_major_locator(locator)
                            x = list(completableProgressIncreaseDecrease[player].keys())
                        plt.scatter(x,y, label=player)
                        plt.plot(x,y)
                    #legend graph and give a date format
                    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
                    #orient xticks
                    plt.xticks(rotation=45, ha="right")
                    #display title and labels
                    if absoluteToRelativeCompletableProgress.value:
                        plt.xlabel('Time (in minutes)')
                    else:
                        plt.xlabel('Date')
                    plt.ylabel("Progress increase/decrease")
                    plt.title("Progress increase/decrease of players for "+completable)
        plt.show()
        accordionCompletableProgressIncreaseDecrease.observe(on_selected_index_completable_progress_increase_decrease_change, 'selected_index')
        accordionCompletableProgressIncreaseDecrease.selected_index=0
    else:
        #create new fig with custom name in the tab
        name=os.path.splitext(filename)[0]+"_completable_progress_increase_decrease"
        fig = plt.figure(num=name, constrained_layout=True)
        fig.canvas.header_visible = False
        fig.canvas.layout.min_height = '400px'
        figCompletableProgressIncreaseDecrease.append(fig)
        noDataToFillVis(10)
        #display title and labels
        plt.xlabel("Time")
        plt.ylabel("Progress of this completable")
        plt.title("Progress increase/decrease of players per completable")
        plt.show()
    with outputCompletableProgressIncreaseDecreaseUpdate:
        clear_output(wait=False)
